In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "KGAT_17c43012d9e77edf2c183a25acb1489b"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
os.system(f"rm -rf {REPO}")
os.system(f"git clone https://github.com/vincentvdo6/competition_package.git {REPO}")
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

os.system('ln -sf /content/data/train.parquet datasets/train.parquet')
os.system('ln -sf /content/data/valid.parquet datasets/valid.parquet')

assert os.path.exists("datasets/train.parquet")
assert os.path.exists("datasets/valid.parquet")
print("Commit:", subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip())
print(f"GPU: {os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train baseline_match × 3 seeds (CONTROL for all comparisons)
# h=64, 3 layers, 32 raw features, linear output — matches official baseline arch
import os, subprocess
os.chdir("/content/competition_package")

for seed in [42, 43, 44]:
    print(f"\n{'='*60}")
    print(f'Training gru_baseline_match_v1 seed {seed}')
    print(f"{'='*60}", flush=True)
    p = subprocess.Popen(
        ['python', '-u', 'scripts/train.py',
         '--config', 'configs/gru_baseline_match_v1.yaml',
         '--seed', str(seed), '--device', 'cuda'],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    for line in p.stdout:
        print(line, end='')
    rc = p.wait()
    if rc != 0:
        print(f'ERROR: seed {seed} failed with exit code {rc}')

print('\nBaseline match training done!')

In [ ]:
# Cell 3: WINDOWED INFERENCE EVAL — key test!
# The official baseline uses 100-step context window, not step-by-step.
# Test if windowed inference improves our baseline_match checkpoints.
import os, subprocess, glob
os.chdir("/content/competition_package")

for pt in sorted(glob.glob('logs/gru_baseline_match_v1_seed*.pt')):
    if '_epoch' in pt:
        continue
    seed = pt.split('seed')[1].split('.')[0]
    npz = f'logs/normalizer_gru_baseline_match_v1_seed{seed}.npz'
    print(f"\n{'='*60}")
    print(f'Windowed eval: seed {seed}')
    print(f"{'='*60}", flush=True)
    p = subprocess.Popen(
        ['python', '-u', 'scripts/eval_windowed.py',
         '--checkpoint', pt, '--normalizer', npz,
         '--window', '100'],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    for line in p.stdout:
        print(line, end='')
    rc = p.wait()
    if rc != 0:
        print(f'ERROR: windowed eval failed with exit code {rc}')

In [ ]:
# Cell 4: Train chrono v2 × 3 seeds (FIXED: bias_hh only, T=10)
import os, subprocess
os.chdir("/content/competition_package")

for seed in [42, 43, 44]:
    print(f"\n{'='*60}")
    print(f'Training gru_v2_chrono seed {seed}')
    print(f"{'='*60}", flush=True)
    p = subprocess.Popen(
        ['python', '-u', 'scripts/train.py',
         '--config', 'configs/gru_v2_chrono.yaml',
         '--seed', str(seed), '--device', 'cuda'],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    for line in p.stdout:
        print(line, end='')
    rc = p.wait()
    if rc != 0:
        print(f'ERROR: seed {seed} failed with exit code {rc}')

print('\nChrono v2 training done!')

In [ ]:
# Cell 5: Train aug v2 × 3 seeds (FIXED: scale 0.95-1.05)
import os, subprocess
os.chdir("/content/competition_package")

for seed in [42, 43, 44]:
    print(f"\n{'='*60}")
    print(f'Training gru_v2_aug seed {seed}')
    print(f"{'='*60}", flush=True)
    p = subprocess.Popen(
        ['python', '-u', 'scripts/train.py',
         '--config', 'configs/gru_v2_aug.yaml',
         '--seed', str(seed), '--device', 'cuda'],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    for line in p.stdout:
        print(line, end='')
    rc = p.wait()
    if rc != 0:
        print(f'ERROR: seed {seed} failed with exit code {rc}')

print('\nAugmentation v2 training done!')

In [ ]:
# Cell 6: Train SWA v2 × 3 seeds (FIXED: start_epoch=15)
import os, subprocess
os.chdir("/content/competition_package")

for seed in [42, 43, 44]:
    print(f"\n{'='*60}")
    print(f'Training gru_v2_swa seed {seed}')
    print(f"{'='*60}", flush=True)
    p = subprocess.Popen(
        ['python', '-u', 'scripts/train.py',
         '--config', 'configs/gru_v2_swa.yaml',
         '--seed', str(seed), '--device', 'cuda'],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1
    )
    for line in p.stdout:
        print(line, end='')
    rc = p.wait()
    if rc != 0:
        print(f'ERROR: seed {seed} failed with exit code {rc}')

print('\nSWA v2 training done!')

In [ ]:
# Cell 7: Evaluate ALL configs
import os, torch, glob
os.chdir("/content/competition_package")

configs_to_eval = [
    ('baseline_match', 'gru_baseline_match_v1'),
    ('chrono_v2', 'gru_v2_chrono'),
    ('aug_v2', 'gru_v2_aug'),
    ('swa_v2', 'gru_v2_swa'),
]

all_results = {}
for label, prefix in configs_to_eval:
    print(f'\n--- {label} ({prefix}) ---')
    scores = []
    for pt in sorted(glob.glob(f'logs/{prefix}_seed*.pt')):
        if '_epoch' in pt:
            continue
        ckpt = torch.load(pt, map_location='cpu', weights_only=False)
        score = float(ckpt.get('best_score', 0.0))
        epoch = ckpt.get('best_epoch', 0)
        print(f'  {os.path.basename(pt)}: val={score:.4f} (best epoch {epoch})')
        scores.append(score)
    if scores:
        mean_score = sum(scores) / len(scores)
        print(f'  Mean: {mean_score:.4f}, Min: {min(scores):.4f}, Max: {max(scores):.4f}')
        all_results[label] = {'scores': scores, 'mean': mean_score}
    else:
        print('  No checkpoints found!')
        all_results[label] = {'scores': [], 'mean': 0}

print(f"\n{'='*60}")
print('KILL TEST SUMMARY')
print(f"{'='*60}")
bm = all_results.get('baseline_match', {})
bm_mean = bm.get('mean', 0)
bm_scores = bm.get('scores', [])
print(f'baseline_match mean: {bm_mean:.4f} (control)')
print()

for label in ['chrono_v2', 'aug_v2', 'swa_v2']:
    r = all_results.get(label, {})
    r_mean = r.get('mean', 0)
    r_scores = r.get('scores', [])
    delta = r_mean - bm_mean
    n_pos = sum(1 for s, b in zip(r_scores, bm_scores) if s > b) if r_scores and bm_scores else 0
    print(f'{label}: mean={r_mean:.4f}, delta={delta:+.4f}, positive={n_pos}/{len(r_scores)}')
    if delta >= 0.0010 and n_pos >= 2:
        print(f'  PASS!')
    elif delta > 0:
        print(f'  MARGINAL')
    else:
        print(f'  FAIL')

In [ ]:
# Cell 8: Training curves
import json, glob, os
os.chdir("/content/competition_package")

for prefix in ['gru_baseline_match_v1', 'gru_v2_chrono', 'gru_v2_aug', 'gru_v2_swa']:
    print(f'\n--- {prefix} ---')
    for hist_file in sorted(glob.glob(f'logs/training_history_{prefix}*.json')):
        with open(hist_file) as f:
            hist = json.load(f)
        name = os.path.basename(hist_file).replace('training_history_', '').replace('.json', '')
        scores = [s['avg'] for s in hist['val_scores']]
        t0s = [s['t0'] for s in hist['val_scores']]
        t1s = [s['t1'] for s in hist['val_scores']]
        bi = scores.index(max(scores))
        print(f'  {name}: {len(scores)} epochs, best={max(scores):.4f} @{bi+1}, '
              f't0={t0s[bi]:.4f} t1={t1s[bi]:.4f} ratio={t0s[bi]/max(t1s[bi],1e-8):.2f}')

In [ ]:
# Cell 9: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs('logs/slim', exist_ok=True)

for prefix in ['gru_baseline_match_v1', 'gru_v2_chrono', 'gru_v2_aug', 'gru_v2_swa']:
    for pt in sorted(glob.glob(f'logs/{prefix}_*.pt')):
        if '_epoch' in pt:
            continue
        ckpt = torch.load(pt, map_location='cpu', weights_only=False)
        slim = {
            'model_state_dict': ckpt['model_state_dict'],
            'config': ckpt.get('config', {}),
            'best_score': ckpt.get('best_score', None),
        }
        out = f'logs/slim/{os.path.basename(pt)}'
        torch.save(slim, out)
        orig = os.path.getsize(pt) / 1e6
        new = os.path.getsize(out) / 1e6
        print(f'{os.path.basename(pt)}: {orig:.1f}MB -> {new:.1f}MB')
    for npz in sorted(glob.glob(f'logs/normalizer_{prefix}*.npz')):
        shutil.copy(npz, f'logs/slim/{os.path.basename(npz)}')
        print(f'Copied {os.path.basename(npz)}')

print(f'\n--- logs/slim/ ({len(os.listdir("logs/slim"))} files) ---')
total_mb = 0
for f in sorted(os.listdir('logs/slim')):
    sz = os.path.getsize(f'logs/slim/{f}') / 1e6
    total_mb += sz
    print(f'  {f}: {sz:.1f}MB')
print(f'  Total: {total_mb:.1f}MB')

shutil.make_archive('/content/baseline_match_v2', 'zip', '/content/competition_package/logs/slim')
sz = os.path.getsize('/content/baseline_match_v2.zip') / 1e6
print(f'\nbaseline_match_v2.zip: {sz:.1f}MB')

shutil.copy('/content/baseline_match_v2.zip', '/content/drive/MyDrive/wunderfund/baseline_match_v2.zip')
print('Saved to Drive!')